### Loading the necessary libraries

In [1]:
import pandas as pd
%matplotlib inline
import seaborn as sns
import joblib
from imblearn.over_sampling import RandomOverSampler
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from imblearn.over_sampling import RandomOverSampler
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from matplotlib.colors import ListedColormap
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import mutual_info_classif
import numpy as np
import re
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, f1_score,recall_score,precision_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import RandomOverSampler
from sklearn.preprocessing import StandardScaler


### Loading the new unseen dataset

In [2]:
new_data = pd.read_csv('new_unseen_dataset.csv')

# Checking for Missing values
print("Null values:")
print(new_data.isnull().sum())

Null values:
Dis No                                 0
Year                                   0
Seq                                    0
Glide                              13063
Disaster Group                         0
Disaster Subgroup                      0
Disaster Type                          0
Disaster Subtype                    2747
Disaster Subsubtype                13600
Event Name                         10999
Country                                0
ISO                                    0
Region                                 0
Continent                              0
Location                            1346
Origin                             10864
Associated Dis                     11412
Associated Dis2                    13946
OFDA Response                      13194
Appeal                             12204
Declaration                        11517
Aid Contribution                   13967
Dis Mag Value                      10075
Dis Mag Scale                       1073
Lat

### Preprocessing the new unseen dataset

In [3]:
new_data.replace('nan', np.nan, inplace=True)

numerical_cols = new_data.select_dtypes(include=np.number).columns
imputer = SimpleImputer(strategy='mean')
new_data[numerical_cols] = imputer.fit_transform(new_data[numerical_cols])

categorical_cols = new_data.select_dtypes(include='object').columns
imputer = SimpleImputer(strategy='most_frequent')
new_data[categorical_cols] = imputer.fit_transform(new_data[categorical_cols])

# Label encoding for categorical variables
label_encoder = LabelEncoder()
for col in categorical_cols:
    new_data[col] = label_encoder.fit_transform(new_data[col])

### Feature Selection and Engineering - based on domain and mutal knowledge

In [4]:
# Feature selection using mutual information
selected_features = ['Year', 'Dis Mag Scale', 'Dis Mag Value', 'Country', 'Longitude', 'Latitude', 'Disaster Type']
X_selected = new_data[selected_features]

# Saving the selected features into a new CSV file
X_selected.to_csv('new_unseen_preprocessed_data.csv', index=False)

# Loading the preprocessed_data.csv file
new_data_selected = pd.read_csv('new_unseen_preprocessed_data.csv')

# Displaying the first 10 rows of the loaded data
new_data_selected.head(10)

,Year,Dis Mag Scale,Dis Mag Value,Country,Longitude,Latitude,Disaster Type
0,1970.0,0,50935.321952,7,2039,1113,5
1,1970.0,1,50935.321952,9,2039,1113,11
2,1970.0,0,50935.321952,20,2039,1113,5
3,1970.0,1,50935.321952,15,2039,1113,11
4,1970.0,1,50935.321952,15,2039,1113,11
5,1970.0,0,50935.321952,15,2039,1113,5
6,1970.0,1,50935.321952,15,2039,1113,11
7,1970.0,0,50935.321952,26,2039,1113,1
8,1970.0,0,50935.321952,26,2039,1113,5
9,1970.0,0,50935.321952,16,2039,1113,5


### Testing the Saved Model

In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
import joblib

# Load the oversampler and the trained Random Forest model

loaded_model = joblib.load('random_forest_model.joblib')

# Assuming 'new_data.csv' is your new, unseen data
new_data = pd.read_csv('new_unseen_preprocessed_data.csv')

# Separate feature set and target variable
X_new = new_data.drop('Disaster Type', axis=1)
y_new = new_data['Disaster Type']

oversampler = RandomOverSampler()
X_new_resampled, y_new_resampled = oversampler.fit_resample(X_new, y_new)

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X_new_resampled, y_new_resampled, test_size=0.2, random_state=42)

# Standardizing/Scaling the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Standardizing/Scaling the features using the previously defined scaler
X_new_scaled = scaler.transform(X_new_resampled)

# Training the Random Forest model on the resampled data
loaded_model.fit(X_train_scaled, y_train)

# Making predictions using the trained model
predictions = loaded_model.predict(X_test_scaled)

#print("Classification Report on Test Set:")
#print(classification_report(y_test, predictions))

# Making predictions on the whole resampled dataset
predictions_full = loaded_model.predict(X_new_scaled)

print("\nClassification Report")
print(classification_report(y_new_resampled, predictions_full))


Classification Report
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5272
           1       0.90      0.90      0.90      5272
           2       1.00      1.00      1.00      5272
           3       1.00      1.00      1.00      5272
           4       1.00      1.00      1.00      5272
           5       0.99      0.75      0.85      5272
           6       1.00      1.00      1.00      5272
           7       1.00      1.00      1.00      5272
           8       0.94      0.98      0.96      5272
           9       0.84      0.85      0.84      5272
          10       0.93      1.00      0.96      5272
          11       1.00      1.00      1.00      5272
          12       0.91      0.98      0.94      5272
          13       0.91      0.96      0.94      5272

    accuracy                           0.96     73808
   macro avg       0.96      0.96      0.96     73808
weighted avg       0.96      0.96      0.96     73808

